## Pandas analysis

In the following a series of exercises is proposed on a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGA's. Each measurement (i.e. each raw) consists of the address of the TDC providing the signal, 'FPGA' and 'TDC_Channel, and the timing information itself, 'ORBIT_CNT', 'BX_COUNTER' and 'TDC_MEAS'. Each TDC count correspond 25/30 ns, whereas the BX_COUNTER feauters gets updated every 25 ns and the ORBIT_CNT every 'x' BX_COUNTER. You can see these way of storing the time as similar to hours, minutes and seconds.

1\. Create a Pandas DataFrame by read N raws of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of raws and larger that 10k.

2\. Find out the value of 'x'

3\. Find out how much the data taking lasted. You can either make an estimate on the baseis of the fraction of the measurements (raws) you read, or perform this check precisely by reading out the whole dataset

4\. Create a new column with the actual time in ns (as a combination of the other three columns with timing information)

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1

6\. Create a new DataFrame with only the raws with HEAD=1

7\. Make two occupancy plots (one per FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [1]:
import pandas as pd
import numpy as np

file_name="./data_000637.txt"
n_rows=1500000
if n_rows>1310720:
    n_rows=1310720
if n_rows<10000:
    n_rows=10000
data=pd.read_csv(file_name)
print("1) \n")
data

1) 



,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14
1310716,1,1,4,3869211171,763,11
1310717,1,0,64,3869211171,764,0
1310718,1,0,139,3869211171,769,0


In [2]:
#2
print("2) \n")
#data.sort_values(by="BX_COUNTER", ascending=False)
#x e' il massimo di BX_COUNTER, cerca metodo
x=data['BX_COUNTER'].max(axis=0)+1
print(data.max(axis=0))
#data

2) 

HEAD                    1
FPGA                    1
TDC_CHANNEL           139
ORBIT_CNT      3869211171
BX_COUNTER           3563
TDC_MEAS               29
dtype: int64


In [3]:
#3 and 4
print("3) and 4) \n")
data['time_ns']=data['TDC_MEAS']*25/30+data['BX_COUNTER']*25+data['ORBIT_CNT']*x*25

time_total=data['time_ns'].max(axis=0)-data['time_ns'].min(axis=0)
print(time_total)
data['time_from_0']=data['time_ns']-data['time_ns'].min(axis=0)
#print(data['time_ns'][0])
data

3) and 4) 

980416253.3125


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time_ns,time_from_0
0,1,0,123,3869200167,2374,26,3.447457e+14,0.000000e+00
1,1,0,124,3869200167,2374,27,3.447457e+14,8.125000e-01
2,1,0,63,3869200167,2553,28,3.447457e+14,4.476625e+03
3,1,0,64,3869200167,2558,19,3.447457e+14,4.594125e+03
4,1,0,64,3869200167,2760,25,3.447457e+14,9.649125e+03
...,...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,3.447467e+14,9.804161e+08
1310716,1,1,4,3869211171,763,11,3.447467e+14,9.804161e+08
1310717,1,0,64,3869211171,764,0,3.447467e+14,9.804161e+08
1310718,1,0,139,3869211171,769,0,3.447467e+14,9.804163e+08


In [4]:
print("5) \n")
#5
data['HEAD']=np.random.randint(0,2,len(data['HEAD']))
data

5) 



,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time_ns,time_from_0
0,1,0,123,3869200167,2374,26,3.447457e+14,0.000000e+00
1,0,0,124,3869200167,2374,27,3.447457e+14,8.125000e-01
2,1,0,63,3869200167,2553,28,3.447457e+14,4.476625e+03
3,0,0,64,3869200167,2558,19,3.447457e+14,4.594125e+03
4,1,0,64,3869200167,2760,25,3.447457e+14,9.649125e+03
...,...,...,...,...,...,...,...,...
1310715,1,0,62,3869211171,762,14,3.447467e+14,9.804161e+08
1310716,0,1,4,3869211171,763,11,3.447467e+14,9.804161e+08
1310717,0,0,64,3869211171,764,0,3.447467e+14,9.804161e+08
1310718,0,0,139,3869211171,769,0,3.447467e+14,9.804163e+08


In [5]:
print("6) \n")
data_6=data[data['HEAD']==1]
data_6

6) 



,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time_ns,time_from_0
0,1,0,123,3869200167,2374,26,3.447457e+14,0.000000e+00
2,1,0,63,3869200167,2553,28,3.447457e+14,4.476625e+03
4,1,0,64,3869200167,2760,25,3.447457e+14,9.649125e+03
5,1,0,63,3869200167,2762,4,3.447457e+14,9.681625e+03
7,1,0,139,3869200167,2776,0,3.447457e+14,1.002831e+04
...,...,...,...,...,...,...,...,...
1310711,1,1,39,3869211171,430,0,3.447467e+14,9.804078e+08
1310713,1,0,64,3869211171,758,18,3.447467e+14,9.804160e+08
1310714,1,0,60,3869211171,762,2,3.447467e+14,9.804161e+08
1310715,1,0,62,3869211171,762,14,3.447467e+14,9.804161e+08


In [6]:
import matplotlib.pyplot as plt
print("7) \n")

TDC_FPGA_0= data[data['FPGA']==0]['TDC_CHANNEL']
TDC_FPGA_1= data[data['FPGA']==1]['TDC_CHANNEL']

plt.figure(figsize=(10,6))
plt.hist(TDC_FPGA_0, label='FPGA_0')
plt.hist(TDC_FPGA_1, label='FPGA_1')
plt.xlabel("TDC_CHANNEL")
plt.ylabel("counts")
plt.legend()
plt.show()


7) 



<Figure size 1000x600 with 1 Axes>

In [7]:
print("8) \n")
FPGA_0_noisy_ch=data[data['FPGA']==0][['TDC_CHANNEL', 'time_ns']].groupby('TDC_CHANNEL').count().nlargest(3,'time_ns')
print("*****FPGA_0 *****\n", FPGA_0_noisy_ch)

FPGA_1_noisy_ch=data[data['FPGA']==1][['TDC_CHANNEL', 'time_ns']].groupby('TDC_CHANNEL').count().nlargest(3,'time_ns')
print("****FPGA_1 ***** \n", FPGA_1_noisy_ch)

#groupby TDC, then count how many values in time_ns column, then take 3 largest

8) 

*****FPGA_0 *****
              time_ns
TDC_CHANNEL         
139            75617
64             64581
63             63724
****FPGA_1 ***** 
              time_ns
TDC_CHANNEL         
2              32669
139            32442
1              28438


In [8]:
print("9) \n")
print("Number of unique orbits: ", data["ORBIT_CNT"].nunique())
print("Number of unique orbits channel 139: ", data[data["TDC_CHANNEL"]==139]["ORBIT_CNT"].nunique())

9) 

Number of unique orbits:  11001
Number of unique orbits channel 139:  10976
